Back Tracing is the processing of reverse engineering a function back to the original formula. Most researchers do this to understand which part of the mathematical foundations of the algorithmic function can be further improved.

Example, you are given a code:

In [1]:
def euclidean_distance(x1,x2,y1,y2):
    z = ((x1-x2)**2+(y1-y2)**2)**(0.5)
    return z

The formula above is a function for the Euclidean distance, back tracing (reverse engineering) it will yield:

$$
z = \sqrt{(x_1-x_2)^2 + (y_1 - y_2)^2}
$$

## Back tracing of the functions/methods from the Multionomial Expectation Maximizer 
Repository codebase by Adrien Biarnes: https://github.com/biarne-a/MNMM .

Back tracing is mostly based on his medium article about the codebase Multinomial Mixture Model and its application in supermarket shoppers' segmentation: 

https://towardsdatascience.com/multinomial-mixture-model-for-supermarket-shoppers-segmentation-a-complete-tutorial-268974d905da

In [ ]:
model = MultinomialExpectationMaximizer(k, restarts=10, rtol=1e-3)

def __init__(self, K, rtol=1e-3, max_iter=100, restarts=10):
        self._K = K
        self._rtol = rtol
        self._max_iter = max_iter
        self._restarts = restarts
        

The MultinomialExpectationMaximizer model object is intialized with the following **Parameters:**

- **k:**
    - Number of mixture components
- **rtol:**
    - Used for convergence checking.
    - Stops the algorithm if the improvement in loss between iterations becomes too small.
- **max_iter:**
    - Sets a maximum number of iterations to prevent infinite running.
- **restarts:**
    - Specifies the number of times to restart the Expectation-Maximization (EM) procedure with different initializations.
    - Helps find a better solution by exploring different starting points.

In [ ]:
    def _init_params(self, X):
        C = X.shape[1]
        weights = np.random.randint(1, 20, self._K)
        alpha = weights / weights.sum()
        beta = dirichlet.rvs([2 * C] * C, self._K)
        return alpha, beta

    def _train_once(self, X):
        '''
        Runs one full cycle of the EM algorithm

        :param X: (N, C), matrix of counts
        :return: The best parameters found along with the associated loss
        '''
        loss = float('inf')
        alpha, beta = self._init_params(X)
        gamma = None

        for it in range(self._max_iter):
            prev_loss = loss
            gamma = self._e_step(X, alpha, beta)
            alpha, beta = self._m_step(X, gamma)
            loss = self._compute_vlb(X, alpha, beta, gamma)
            print('Loss: %f' % loss)
            if it > 0 and np.abs((prev_loss - loss) / prev_loss) < self._rtol:
                    break
        return alpha, beta, gamma, loss

## _train_once
_train_once method will run one full cycle of iterations for the EM algorithm.
It stars off initializing the parameters randomly (to avoid getting stuck in dead ends) with the beta parameters being initialized using the Dirichlet distribution which is conjugate to the multinomial distribution. This would produce a set of parameters that sum up to 1 (for every cluster) as required.

To summarize each iteration:
- It calculates probabilities of data points belonging to each cluster through **_e_step_**
- Updates cluster weights and multinomial parameters based on responsibilities through **_m_step_**.
- Tracks and outputs loss since the Expectation-Maximization (EM) algorithm is trying to maximize the lower bound through **_compute_vlb_**
- Stops when either loss improvement is mininal (the _rtol_ parameter) or maximum iterations are reached (the _restarts_ parameter)

In [12]:
def _e_step(self, X, alpha, beta):
    """
    Performs E-step on MNMM model
    Each input is numpy array:
    X: (N x C), matrix of counts
    alpha: (K) or (NxK) in the case of individual weights, mixture component weights
    beta: (K x C), multinomial categories weights

    Returns:
    gamma: (N x K), posterior probabilities for objects clusters assignments
    """
    # Compute gamma
    N = X.shape[0]
    K = beta.shape[0]
    weighted_multi_prob = np.zeros((N, K))
    for k in range(K):
        weighted_multi_prob[:, k] = self._get_mixture_weight(alpha, k) * self._multinomial_prob(X, beta[k])

    # To avoid division by 0
    weighted_multi_prob[weighted_multi_prob == 0] = np.finfo(float).eps

    denum = weighted_multi_prob.sum(axis=1)
    gamma = weighted_multi_prob / denum.reshape(-1, 1)

    return gamma

#### _e_step_
The e_step method (Expectation step) computes a matrix of responsibilities, denoted as gamma (γ), where γ_ik represents the probability that data point i belongs to cluster k. This is calculated using the method's parameters: alpha (current mixture weights ) and beta (multinomial parameters ) for each cluster.

\begin{align*}
\text{weightedmultiprob}_{ik} &= \text{getmixtureweight}(\alpha, k) \times \text{multinomialprob}(X, \beta_k) \quad \text{for } i = 1, 2, \ldots, N \text{ and } k = 1, 2, \ldots, K \\
weightedmultiprob[ \text{weightedmultiprob} = 0] &= \epsilon  \\
\text{denum}_i &= \sum_{k=1}^{K} \text{weightedmultiprob}_{ik} \quad \text{for } i = 1, 2, \ldots, N \\
\gamma_{ik} &= \frac{\text{weightedmultiprob}_{ik}}{\text{denum}_i} \quad \text{for } i = 1, 2, \ldots, N \text{ and } k = 1, 2, \ldots, K
\end{align*}


In [ ]:
    def _m_step(self, X, gamma):
        """
        Performs M-step on MNMM model
        Each input is numpy array:
        X: (N x C), matrix of counts
        gamma: (N x K), posterior probabilities for objects clusters assignments

        Returns:
        alpha: (K), mixture component weights
        beta: (K x C), mixture categories weights
        """
        # Compute alpha
        alpha = self._m_step_alpha(gamma)

        # Compute beta
        beta = self._m_step_beta(X, gamma)

        return alpha, beta

    def _m_step_alpha(self, gamma):
        alpha = gamma.sum(axis=0) / gamma.sum()
        return alpha

    def _m_step_beta(self, X, gamma):
        weighted_counts = gamma.T.dot(X)
        beta = weighted_counts / weighted_counts.sum(axis=-1).reshape(-1, 1)
        return beta

#### _m_step_
The m_step method (Maximation step) update the model parameters to maximize the expected complete-data log-likelihood, which was calculated in the E-step.

For alpha, the new mixture weight for cluster k is calculated by averaging the responsibilities γ_ik across all data points i.

\begin{align*}
\alpha_k &= \frac{\sum_{i=1}^{N} \gamma_{ik}}{N} \quad \text{for } k = 1, 2, \ldots, K
\end{align*}

For beta, the new multinomial parameter for category j in cluster k is calculated by taking a weighted average of the corresponding counts X_ij across all data points i, weighted by their responsibilities γ_ik.

\begin{align*}
\beta_{kj} &= \frac{\sum_{i=1}^{N} \gamma_{ik} X_{ij}}{\sum_{i=1}^{N} \gamma_{ik}} \quad \text{for } k = 1, 2, \ldots, K \text{ and } j = 1, 2, \ldots, C
\end{align*}

In [ ]:
def _compute_vlb(self, X, alpha, beta, gamma):
        """
        Computes the variational lower bound
        X: (N x C), data points
        alpha: (K) or (NxK) with individual weights, mixture component weights
        beta: (K x C), multinomial categories weights
        gamma: (N x K), posterior probabilities for objects clusters assignments

        Returns value of variational lower bound
        """
        loss = 0
        for k in range(beta.shape[0]):
            weights = gamma[:, k]
            loss += np.sum(weights * (np.log(self._get_mixture_weight(alpha, k)) + self._multinomial_prob(X, beta[k], log=True)))
            loss -= np.sum(weights * np.log(weights))
        return loss

#### _compute_vlb_
Calculates the variational lower bound (VLB), basically the loss, for the MMM.

\begin{align*}
\text{loss} = &\sum_{k=1}^{K} \left[ \sum_{i=1}^{N} \gamma_{ik} \left( \log(\text{\_get\_mixture\_weight}(\alpha, k)) + \text{\_multinomial\_prob}(X_i, \beta_k, \text{log=True}) \right) \right] \\
&- \sum_{i=1}^{N} \gamma_{ik} \log(\gamma_{ik})
\end{align*}


In [ ]:
def _multinomial_prob(self, counts, beta, log=False):
        """
        Evaluates the multinomial probability for a given vector of counts
        counts: (N x C), matrix of counts
        beta: (C), vector of multinomial parameters for a specific cluster k
        Returns:
        p: (N), scalar values for the probabilities of observing each count vector given the beta parameters
        """
        n = counts.sum(axis=-1)
        m = multinomial(n, beta)
        if log:
            return m.logpmf(counts)
        return m.pmf(counts)

The method multinomial_prob, as the comment say, evaluates the multinomial probability for a given vector of counts

$$
log(p_{i})= \log(Multinomial(counts_{i}|n_{i},\beta))
$$

In [9]:
def compute_log_likelihood(self, X_test, alpha, beta):
        mn_probs = np.zeros(X_test.shape[0])
        for k in range(beta.shape[0]):
            mn_probs_k = self._get_mixture_weight(alpha, k) * self._multinomial_prob(X_test, beta[k])
            mn_probs += mn_probs_k
        mn_probs[mn_probs == 0] = np.finfo(float).eps
        return np.log(mn_probs).sum()

#### _compute_log_likelihood_

Calculates the log-likelihood of a given test dataset under a Multinomial Mixture Model using the mixture weights of the MMM (_alpha_) and the component probability distributions (_beta_).

$$
Computeloglikelihood(X_{test}, \alpha,\beta) = \sum_{i=1}^{|X_{test}|} \log (\sum_{k=1}^{|\beta|} GetMixtureWeight(\alpha, k). MultinomialProb(X_{test[i]}, \beta_{k}))
$$
    
It returns the log-likelihood of the entire test data under the MMM, calculated as the sum of the log of the mixture probabilities.

In [10]:
def compute_bic(self, X_test, alpha, beta, log_likelihood=None):
        if log_likelihood is None:
            log_likelihood = self.compute_log_likelihood(X_test, alpha, beta)
        N = X_test.shape[0]
        return np.log(N) * (alpha.size + beta.size) - 2 * log_likelihood


#### _compute_bic_
Returns Bayesian Information Criterion (BIC) of the MMM using the mixture weights of the MMM (_alpha_) and the component probability distributions (_beta_) and pre-computed log-likelihood if available.
$$
BIC = -2.computeloglikelihood(x_{test},\alpha, \beta) + \log(N) . (k_{\alpha}+ k_{\beta})
$$

In [ ]:
def compute_icl_bic(self, bic, gamma):
        classification_entropy = -(np.log(gamma.max(axis=1))).sum()
        return bic - classification_entropy

#### _compute_icl_bic_
Calculates the Integrated Complete-data Likelihood (ICL) BIC for the MMM using the BIC from _compute_bic_ the method and gamma
$$
ICL = BIC - \sum_{i=1}^{N} \log (max_{j})\gamma_{ij}
$$